* Make object list for Tabletop env refered from language table simulation dataset

In [ ]:
domain= "Tabletop"

color_list= ['green', 'red', 'yellow', 'blue']
shape_list= ['star', 'moon', 'crescent', 'pentagon', 'square', 'circle', 'semicircle']
block= 'block'
cube= 'cube'
bowl= "bowl"

In [ ]:
objs_list= list()

for color in color_list:
    for shape in shape_list:
        _obj= color + ' ' + shape + ' ' + block
        objs_list.append(_obj)
    _obj= color + ' '+ cube
    objs_list.append(_obj)
    
    _obj= color + ' '+ bowl
    objs_list.append(_obj)

In [ ]:
objs_dict= dict()
objs_dict['place']= domain
objs_dict['objects']= objs_list

In [ ]:
file_path= './datasets/{}/objects.json'.format(domain)
with open(file_path, 'w') as json_file:
    json.dump(objs_dict, json_file)

---

In [ ]:
import openai
import json
import re
import copy
import os
import random

In [ ]:
domain= "Tabletop"
OPENAI_API_KEY = # input your openai key
openai.api_key = OPENAI_API_KEY
gpt_model = "gpt-3.5-turbo-1106"

In [ ]:
# Remove new objects that are out of object list
def split_object_inst(org_objs, high_insts_list):    
    ins_objs = list()
    only_high_insts = list()
    noise_prompt= list()
    out_of_objs= False

    for idx, high_inst in enumerate(high_insts_list): # each instruction
        
        parts = high_inst.strip()[1:].split(')')
        out_of_objs= False
        
        if len(parts) != 2:
            print("There is abnormal format at all: ", high_inst)
            noise_prompt.append(idx)
            continue
            
        # objects
        _ins_objs= parts[0].split(',')
        
        if len(_ins_objs)< 2:
            print("There is abnormal format in object: ", high_inst)
            noise_prompt.append(idx)
            continue
        
        for i, _objs in enumerate(_ins_objs):
            _ins_objs[i]= re.sub(r'^[^A-Za-z0-9]+|[^A-Za-z0-9]+$', '', _objs).strip()
        
        # If new objects are used that is not in the objects list, the output is considered noise.
        for _objs in _ins_objs:
            if _objs.lower() not in org_objs:
                print("There is objects that is out of object list: ", _objs)
                out_of_objs= True
                break
                
        if out_of_objs==True:
            noise_prompt.append(idx)
            continue
        
        # commands
        parts[1]= parts[1].strip()
        parts[1]= re.sub(r'^[^A-Za-z0-9]+|[^A-Za-z0-9]+$', '', parts[1])
        only_high_insts.append(parts[1])
        ins_objs.append(_ins_objs)
        
    return ins_objs, only_high_insts, noise_prompt

In [ ]:
# Just split objects and command, and remove noise.
def split_object_inst2(high_insts_list):
    ins_objs = list()
    only_high_insts = list()
    noise_prompt= list()

    for idx, high_inst in enumerate(high_insts_list): # each instruction
        parts = high_inst.strip().split(')')
            
        # objects
        __ins_objs= parts[0].strip()
        _ins_objs= __ins_objs.split(',')

        for i, _objs in enumerate(_ins_objs):
            _ins_objs[i]= re.sub(r'^[^A-Za-z0-9]+|[^A-Za-z0-9]+$', '', _objs.strip()).strip()
        
        # If there are more than 3 tokens, don't consider it an object
        noise= list()
        for obj in _ins_objs:
            if obj.strip() == '':
                _ins_objs.remove(obj)
                continue
                
            token_count= obj.split(" ")
            if len(token_count) > 3:
                _ins_objs.remove(obj)
        
        if _ins_objs==[]:
            noise_prompt.append(idx)
            continue

        ins_objs.append(_ins_objs)

        # instruction
        parts[1]= re.sub(r'^[^A-Za-z0-9]+|[^A-Za-z0-9]+$', '', parts[1].strip()).strip()
        only_high_insts.append(parts[1])

    return ins_objs, only_high_insts, noise_prompt

In [ ]:
file_path= './prompt_templates/fixed_objects/Commands_generating_prompt.txt'
with open(file_path, 'r') as prompt_file:
    command_prompt= prompt_file.read()

In [ ]:
file_path= './prompt_templates/fixed_objects/Action_steps_generating_prompt.txt'
with open(file_path, 'r') as prompt_file:
    steps_prompt= prompt_file.read()

In [ ]:
# Read object list
file_path= './datasets/{}/objects.json'.format(domain)
with open(file_path, 'r') as objects_file:
    block_objects= json.load(objects_file)

----
* Generate Dataset

In [ ]:
commands_dict= dict()
commands_dict['place']= domain
commands_dict['objects']= block_objects['objects']

sample_input1= '(Red cube, Yellow cube, Blue cube, Green cube), Place a cube of a different color in each corner.'
high_insts_list=[sample_input1]
commands_dict['prompt_output(high)']= high_insts_list

In [ ]:
iter_num=0
iter_continue= True
generate_num= 20
dataset_size= 30000

# First iteration block
print("Start to generate high_instrctions")
while True:
    try:
        ### Iteration stop condition
        if dataset_size <= len(commands_dict['high_instructions']):
            break
            
        iter_num+=1
        _high_insts_list= list()
        
        ### 1. Generate 20 high-level commands using objects list
        random.shuffle(commands_dict['objects'])
        prompt= command_prompt.format(generate_num, commands_dict['objects'], high_insts_list[0])
        messages = [
            {"role": "user", "content": prompt}
        ]  

        _response = openai.ChatCompletion.create(
        model=gpt_model,
        messages=messages)
        
        _output = _response['choices'][0]['message']['content']
        
        # Output post-processing
        _high_insts= _output.strip()
        _high_insts= '2. '+ _high_insts

        sentences = _high_insts.split('\n')
        sentences = [sentence.strip() for sentence in sentences if sentence.strip()]
        _high_insts_list = [re.sub(r'^\d+\.\s*', '', sentence) for sentence in sentences]
        
        for idx, inst in enumerate(_high_insts_list):
            _high_insts_list[idx]= _high_insts_list[idx].strip()

        if '' in _high_insts_list:
            while '' in _high_insts_list:
                _high_insts_list.remove('')
        
        # If len(_high_inst_list) != generate_num, we assume the outputs are noisy and remove last 10 commands.
        if len(_high_insts_list) != generate_num:
            if len(_high_insts_list) != (generate_num-1):
                print('The number of generated commands is not {} : {}.'.format(generate_num, len(_high_insts_list)))
                print(" I expect there is some issue in the output, so I only get the output without last 10 sentences.", end='\n\n')
                if len(_high_insts_list) <= 10:
                    print("the number of generated instruction is less than 10. Pass the generation and continue.")
                    print("current iter_num: ", iter_num)
                    print("=================================", end='\n\n')
                    iter_num-=1
                    continue
                else:
                    _high_insts_list= _high_insts_list[:len(_high_insts_list)-10]
        
        high_insts_list.extend(_high_insts_list)
        commands_dict['prompt_output(high)']= high_insts_list

        if iter_num%100 == 0:
            print("Objects for {} generated (total {}) : {}".format(commands_dict['place'], len(commands_dict['prompt_output(high)']), iter_num))
    
        ### 2. Detect new objects used in generated commands, remove them.
        used_objs, only_high_insts, noise_prompt= split_object_inst(commands_dict['objects'],\
                                                         commands_dict['prompt_output(high)'])
        commands_dict['high_instructions']= only_high_insts # format= list
        commands_dict['used_objects']= used_objs
        
        if len(noise_prompt) >= 1:
            for idx in noise_prompt:
                del commands_dict['prompt_output(high)'][idx]
            print("Delete abnormal format's prompt output")
        print(len(commands_dict['prompt_output(high)']), len(commands_dict['high_instructions']))
        # Must be len(prompt_output) == len(high_instructions)

        ### save file
        with open(file_path+'/Commands.json', 'w') as instructions_file:
            json.dump(commands_dict, instructions_file, indent='\t')

    except Exception as ex:
        print("\'\'\'There is error in first iteration block. Ignore the error and continue the iteration.")
        print("Error message : ", ex)
        print("Error iter_num : {}\'\'\''".format(iter_num), end='\n\n')
        iter_num-=1
        continue

print("Complete to generate high instructions {}".format(len(commands_dict['high_instructions'])))

In [ ]:
dataset_dict= dict()
dataset_dict['place']= domain
dataset_dict['objects']= commands_dict['objects']
_high_inst_list_for_final= list()
steps= list()

gpt_model = "gpt-3.5-turbo-1106"

i=0
total_iter2= len(commands_dict['high_instructions'])
print("total the number of high_instructions: ", total_iter2)
print("Processed idx: ", end=' ')

# second iteration block
while True:
    if i == total_iter2:
        break

    try: 
        objs_on_table, high_inst, _ = split_object_inst2([commands_dict['prompt_output(high)'][i]])
        objs_on_table= objs_on_table[0]
        high_inst= high_inst[0]
        print(objs_on_table, high_inst)

        prompt= steps_prompt.format(objs_on_table, high_inst)
        generate_steps_messages = [
                {"role": "user", "content": prompt}
        ]    

        steps_response = openai.ChatCompletion.create(
            model=gpt_model,
            messages=generate_steps_messages
        )

        _steps_i = steps_response['choices'][0]['message']['content']
        _steps_i=  _steps_i.strip()

        steps.append(_steps_i)
        _high_inst_list_for_final.append(high_inst)

        dataset_dict['prompt_output(steps)']= steps
        dataset_dict['high_instructions']= _high_inst_list_for_final

        with open(file_path+'/_Dataset.json', 'w') as save_file:
            json.dump(dataset_dict, save_file, indent='\t')

        i+=1
        print(i, end=' ')

    except Exception as ex:
        print("\'\'\'There is error in second iteration block. Ignore the error and continue the iteration.")
        print("Error message : ", ex)
        print("Error iter_num : {}\'\'\''".format(i), end='\n\n')
        continue

print("Complete generating steps for all high_instructions ", end=' ')
print(len(dataset_dict['high_instructions']), len(dataset_dict['prompt_output(steps)']))
# Must be len(dataset_dict['high_instructions']) == len(dataset_dict['prompt_output(steps)'])